## Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
# import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

In [ ]:
max_semesters_to_display = 8
df_raw = pd.read_csv("student_dummy_data.csv", delimiter=";", encoding="latin-1", decimal=",")
df_reduced = df_raw.loc[df_raw["besuchtesSemester"] <= max_semesters_to_display]
df_by_semester_and_study_grps = df_reduced.groupby(["besuchtesSemester", "STUDBEZ_kompakt"])
df_by_semester_and_study_agg = df_by_semester_and_study_grps["ECTS_Sem"].agg(["mean", "median"])
df_by_semester_and_study_agg = pd.DataFrame(df_by_semester_and_study_agg).reset_index()
df_by_semester_and_study_agg.head()

,besuchtesSemester,STUDBEZ_kompakt,mean,median
0,1.0,BA; BI,8.198052,8.0
1,1.0,BA; PPP,14.431443,14.5
2,1.0,MA; BI,4.500000,3.5
3,1.0,MA; PPP,13.135281,12.0
4,2.0,BA; BI,10.350482,10.0


In [ ]:
fig = px.line(df_by_semester_and_study_agg, 
                x="besuchtesSemester", 
                y="mean", 
                color="STUDBEZ_kompakt", 
                title="<b>Durchschnittliche ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>")
fig.update_xaxes(dtick=1)

# TEST

In [ ]:
# IMPORTS
import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
app = Dash(__name__)

df = df_by_semester_and_study_agg.copy()

available_indicators = df["STUDBEZ_kompakt"].unique()

,besuchtesSemester,STUDBEZ_kompakt,mean,median
0,1.0,BA; BI,8.198052,8.000
1,1.0,BA; PPP,14.431443,14.500
2,1.0,MA; BI,4.500000,3.500
3,1.0,MA; PPP,13.135281,12.000
4,2.0,BA; BI,10.350482,10.000
5,2.0,BA; PPP,16.717071,17.000
6,2.0,MA; BI,6.388889,5.000
7,2.0,MA; PPP,15.191143,15.000
8,3.0,BA; BI,10.301136,10.500
9,3.0,BA; PPP,19.117647,19.500


In [ ]:

# App layout
app.layout = html.Div([
    html.Div([

        html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),
        
        dcc.Dropdown(id="slct_study", # was xaxis-column
                    options=[
                        {"label": i, "value": i} for i in available_indicators],
                    multi=False,
                    value=available_indicators[0],
                    style={'width': '40%'}
        ),
        dcc.RadioItems(
                    id='measure', # was xaxis-type
                    options=[{'label': i, 'value': i} for i in ['mean', 'median']],
                    value='median',
                    labelStyle={'display': 'inline-block'}
        )
    ], style={'width': '40%'}),

    html.Div([

        dcc.Graph(id="mygraph1", figure={}), # was bof

        dcc.Slider(
                id='max_semester_slider', # was year--slider
                min=df['besuchtesSemester'].min(),
                max=df['besuchtesSemester'].max(),
                value=df['besuchtesSemester'].max(),
                marks={str(besuchtesSemester): str(besuchtesSemester) for besuchtesSemester in df['besuchtesSemester'].unique()},
                step=None
        )
    ])
])



In [ ]:
max_semester = 6
chosen_measure = "mean"
name_of_study = "BA; PPP"

dff = df[df["besuchtesSemester"] <= max_semester]
dff = dff[dff["STUDBEZ_kompakt"] == name_of_study]

# plot
fig = px.line(dff, 
                x="besuchtesSemester", 
                y=chosen_measure, 
                #color="STUDBEZ_kompakt", 
                title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>", 
                labels=dict(besuchtesSemester="besuchtes Semester", 
                            mean = "ECTS-Punkte"),
                markers=True)
# remove decimals from x axis
fig.update_xaxes(dtick=1)
# add anotations and hide legend
fig.update_layout(showlegend=False, 
                yaxis_range=[0,25])

In [ ]:
#connecting graph with components
@app.callback(
    Output(component_id="mygraph1", component_property="figure"),
    Input(component_id="slct_study", component_property="value"),
    Input("measure", "value"), 
    Input("max_semester_slider", "value"),
    prevent_initial_call=True
    )
    
def update_graph(name_of_study, chosen_measure, max_semester):

    dff = df[df["besuchtesSemester"] <= max_semester]
    dff = dff[dff["STUDBEZ_kompakt"] == name_of_study]

    # plot
    fig = px.line(dff, 
                    x="besuchtesSemester", 
                    y=chosen_measure, 
                    #color="STUDBEZ_kompakt", 
                    title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>", 
                    labels=dict(besuchtesSemester="besuchtes Semester", 
                                mean = "ECTS-Punkte"),
                    markers=True)
    # remove decimals from x axis
    fig.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig.update_layout(showlegend=False, 
                    yaxis_range=[0,25])
                
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [94]:
def create_lineplot_annotations_y(dataframe, 
                                    max_sem_in_plot,
                                    y_variable="mean", 
                                    group_variable="STUDBEZ_kompakt"):

    """
    This function creates y labels based on 'gorup_variable' for a line plot. 
    The labels are positioned at the height of the right-most y value.
    """
    y_heights = dataframe.loc[dataframe["besuchtesSemester"]==\
        max_sem_in_plot][[y_variable, group_variable]]
    y_heights = y_heights.sort_values(by="mean", ascending=False)
    y_heights_mean = list(y_heights[y_variable])
    y_heights_labs = list(y_heights[group_variable])

    ants = []

    for i in range(0,len(y_heights_labs)):
        ants.append(dict(xref='paper', 
                        x=1, 
                        y=round(y_heights_mean[i]),
                        xanchor='left', yanchor='middle',
                        text=y_heights_labs[i],
                        font={"family": "Arial", 
                        "size": 12},
                        showarrow=False))
    return(ants) # list of annotation dicts

# WORKS

In [101]:
## ----------------------- WORKS!! -----------------------

app = Dash(__name__)

df = df_by_semester_and_study_agg.copy()

available_indicators = df["STUDBEZ_kompakt"].unique()


## ----------------------- LAYOUT -----------------------

app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_study",
                options=[
                        {"label": i, "value": i} for i in available_indicators],
                #  options=[
                #      {"label": "BA; PPP", "value": "BA; PPP"},
                #      {"label": "MA; PPP", "value": "MA; PPP"},
                #      {"label": "BA; BI", "value": "BA; BI"},
                #      {"label": "MA; BI", "value": "MA; BI"}],
                 multi=False,
                 value="BA; PPP",
                 style={'width': "40%"}
    ),
    html.Br(),
    dcc.RadioItems(
                    id='measure', # was xaxis-type
                    options=[{'label': i, 'value': i} for i in ['mean', 'median']],
                    value='mean',
                    labelStyle={'display': 'inline-block'}, 
                    style={'width': '40%'}),
    html.Div(id="output_container", children=[]),
    html.Br(),
    html.Label("Chose maximum semesters to be displayed"),

    dcc.Slider(
                id='max_semester_slider', # was year--slider
                min=df['besuchtesSemester'].min(),
                max=df['besuchtesSemester'].max(),
                value=df['besuchtesSemester'].max(),
                marks={i: str(i) for i in range(1, 9)},
                step=None
    ),

    dcc.Graph(id="bof", figure={}),
    dcc.Graph(id="biff", figure={}),

])


## ----------------------- CALLBACK -----------------------

@app.callback(
    [Output(component_id="output_container", component_property="children"),
     Output(component_id="bof", component_property="figure"),
     Output("biff", "figure")],
    [Input(component_id="slct_study", component_property="value"),
     Input("measure", "value"), 
     Input("max_semester_slider", "value")]
    )
def update_graph(selected_study, selected_measure, max_sem):

    container = "The field of study chosen by user was: {}".format(selected_study)
    
    #selected_measure = selected_measure

    dff = df_by_semester_and_study_agg.copy()
    dff = dff[dff["besuchtesSemester"] <= max_sem]

    # plot
    fig1 = px.line(dff[dff["STUDBEZ_kompakt"] == selected_study], 
                    x="besuchtesSemester", 
                    y=selected_measure, 
                    color="STUDBEZ_kompakt", 
                    title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>",
                    markers=True)
    # remove decimals from x axis
    fig1.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig1.update_layout(showlegend=False, 
                    yaxis_range=[0,25])
    
    antsis = create_lineplot_annotations_y(dataframe=dff, 
                                            max_sem_in_plot = max_sem, 
                                            y_variable=selected_measure, 
                                            group_variable="STUDBEZ_kompakt")

    fig2 = px.line(dff, 
                    x="besuchtesSemester", 
                    y=selected_measure, 
                    color="STUDBEZ_kompakt", 
                    title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>",
                    markers=True)
    fig2.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig2.update_layout(annotations=antsis, 
                    showlegend=False,     
                    yaxis_range=[0,25])

    return container, fig1, fig2


## ------------------------- RUN -------------------------

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

In [97]:
## ----------------------- WORKING -----------------------
app = Dash(__name__)

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_study",
                 options=[
                     {"label": "BA; PPP", "value": "BA; PPP"},
                     {"label": "MA; PPP", "value": "MA; PPP"},
                     {"label": "BA; BI", "value": "BA; BI"},
                     {"label": "MA; BI", "value": "MA; BI"}],
                 multi=False,
                 value="BA; PPP",
                 style={'width': "40%"}
    ),
    html.Br(),
    dcc.RadioItems(
                    id='measure', # was xaxis-type
                    options=[{'label': i, 'value': i} for i in ['mean', 'median']],
                    value='mean',
                    labelStyle={'display': 'inline-block'}, 
                    style={'width': '40%'}),
    html.Div(id="output_container", children=[]),
    html.Br(),
    html.Label("Chose maximum semesters to be displayed"),

    dcc.Slider(
                id='max_semester_slider', # was year--slider
                min=df['besuchtesSemester'].min(),
                max=df['besuchtesSemester'].max(),
                value=df['besuchtesSemester'].max(),
                marks={i: str(i) for i in range(1, 9)},
                step=None
    ),

    dcc.Graph(id="bof", figure={}),

])

# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id="output_container", component_property="children"),
     Output(component_id="bof", component_property="figure")],
    [Input(component_id="slct_study", component_property="value"),
     Input("measure", "value"), 
     Input("max_semester_slider", "value")]
    )
def update_graph(selected_study, selected_measure, max_sem):

    container = "The field of study chosen by user was: {}".format(selected_study)

    dff = df_by_semester_and_study_agg.copy()
    dff = dff[dff["STUDBEZ_kompakt"] == selected_study]
    dff = dff[dff["besuchtesSemester"] <= max_sem]

    # plot
    fig = px.line(dff, 
                    x="besuchtesSemester", 
                    y=selected_measure, 
                    color="STUDBEZ_kompakt", 
                    title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>",
                    markers=True)
    # remove decimals from x axis
    fig.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig.update_layout(showlegend=False, 
                    yaxis_range=[0,25])
                
    return container, fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

In [85]:
## ----------------------- WORKING -----------------------
max_semesters_to_display = 4

app = Dash(__name__)

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_study",
                 options=[
                     {"label": "BA; PPP", "value": "BA; PPP"},
                     {"label": "MA; PPP", "value": "MA; PPP"},
                     {"label": "BA; BI", "value": "BA; BI"},
                     {"label": "MA; BI", "value": "MA; BI"}],
                 multi=False,
                 value="BA; PPP",
                 style={'width': "40%"}
                 ),
    html.Br(),
    dcc.RadioItems(
                    id='measure', # was xaxis-type
                    options=[{'label': i, 'value': i} for i in ['mean', 'median']],
                    value='mean',
                    labelStyle={'display': 'inline-block'}, 
                    style={'width': '40%'}),
    html.Div(id="output_container", children=[]),
    html.Br(),

    dcc.Graph(id="bof", figure={})

])

# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id="output_container", component_property="children"),
     Output(component_id="bof", component_property="figure")],
    [Input(component_id="slct_study", component_property="value"),
     Input("measure", "value")]
    )
def update_graph(selected_study, selected_measure):

    container = "The field of study chosen by user was: {}".format(selected_study)

    dff = df_by_semester_and_study_agg.copy()
    dff = dff[dff["STUDBEZ_kompakt"] == selected_study]
    dff = dff[dff["besuchtesSemester"] <= max_semesters_to_display]

    # plot
    fig = px.line(dff, 
                    x="besuchtesSemester", 
                    y=selected_measure, 
                    color="STUDBEZ_kompakt", 
                    title="<b>ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>",
                    markers=True)
    # remove decimals from x axis
    fig.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig.update_layout(showlegend=False, 
                    yaxis_range=[0,25])
                
    return container, fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

In [48]:
## ----------------------- WORKING -----------------------
max_semesters_to_display = 4

app = Dash(__name__)

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_study",
                 options=[
                     {"label": "BA; PPP", "value": "BA; PPP"},
                     {"label": "MA; PPP", "value": "MA; PPP"},
                     {"label": "BA; BI", "value": "BA; BI"},
                     {"label": "MA; BI", "value": "MA; BI"}],
                 multi=False,
                 value="BA; PPP",
                 style={'width': "40%"}
                 ),
    html.Br(),

    html.Div(id="output_container", children=[]),
    html.Br(),

    dcc.Graph(id="bof", figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id="output_container", component_property="children"),
     Output(component_id="bof", component_property="figure")],
    [Input(component_id="slct_study", component_property="value")]
    )
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The field of study chosen by user was: {}".format(option_slctd)

    dff = df_by_semester_and_study_agg.copy()
    dff = dff[dff["STUDBEZ_kompakt"] == option_slctd]
    dff = dff[dff["besuchtesSemester"] <= max_semesters_to_display]

    # plot
    fig = px.line(dff, 
                    x="besuchtesSemester", 
                    y="mean", 
                    color="STUDBEZ_kompakt", 
                    title="<b>Durchschnittliche ECTS-Punkte nach besuchtem Semester im Studienverlauf</b>", 
                    labels=dict(besuchtesSemester="besuchtes Semester", 
                                mean = "ECTS-Punkte (Durchschnitt)"),
                    markers=True)
    # remove decimals from x axis
    fig.update_xaxes(dtick=1)
    # add anotations and hide legend
    fig.update_layout(showlegend=False, 
                    yaxis_range=[0,25])
                
    return container, fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
BA; PPP
<class 'str'>
